In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch --progress-bar off
!pip install -qqq transformers --progress-bar off
!pip install -qqq langchain --progress-bar off
!pip install -qqq chromadb --progress-bar off
!pip install -qqq pypdf --progress-bar off
!pip install -qqq xformers --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding --progress-bar off
!pip install -qqq pdf2image --progress-bar off

In [2]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

Looking in indexes: https://pypi.org/simple, https://huggingface.github.io/autogptq-index/whl/cu118/
     ---------------------------------------- 0.0/4.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/4.7 MB 1.4 MB/s eta 0:00:04
     ---------------------------------------- 0.0/4.7 MB 393.8 kB/s eta 0:00:12
     ---- ----------------------------------- 0.6/4.7 MB 3.9 MB/s eta 0:00:02
     ------ --------------------------------- 0.7/4.7 MB 3.9 MB/s eta 0:00:01
     -------- ------------------------------- 0.9/4.7 MB 3.9 MB/s eta 0:00:01
     --------- ------------------------------ 1.1/4.7 MB 4.1 MB/s eta 0:00:01
     ---------- ----------------------------- 1.2/4.7 MB 4.1 MB/s eta 0:00:01
     ---------- ----------------------------- 1.2/4.7 MB 4.1 MB/s eta 0:00:01
     ----------- ---------------------------- 1.4/4.7 MB 3.2 MB/s eta 0:00:02
     ------------ --------------------------- 1.5/4.7 MB 3.2 MB/s eta 0:00:02
     -------------- ------------------------- 

In [3]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

C:\Users\abhim\miniforge3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
CUDA extension not installed.
CUDA extension not installed.


In [26]:
loader = PyPDFLoader("C:\\Users\\abhim\\Downloads\\invoice_1.pdf")
docs = loader.load()
len(docs)

1

In [5]:
embeddings = HuggingFaceInstructEmbeddings(
    model_name="hkunlp/instructor-large", model_kwargs={"device": DEVICE}
)

load INSTRUCTOR_Transformer
max_seq_length  512


In [27]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

1

In [28]:
%%time
db = Chroma.from_documents(texts, embeddings, persist_directory="db")

CPU times: total: 11.6 s
Wall time: 1.99 s


In [7]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GPTQ"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(
    model_name_or_path,
    revision="gptq-4bit-128g-actorder_True",
    model_basename=model_basename,
    use_safetensors=True,
    trust_remote_code=True,
    inject_fused_attention=False,
    device=DEVICE,
    quantize_config=None,
)

C:\Users\abhim\miniforge3\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\abhim\.cache\huggingface\hub\models--TheBloke--Llama-2-13B-chat-GPTQ. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
WARNING - Exllamav2 kernel is not installed, reset disable_exllamav2 to True. This may because you installed

In [29]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <>
{system_prompt}
<>

{prompt} [/INST]
""".strip()

In [30]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [10]:
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15,
    streamer=streamer,
)

The model 'LlamaGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCausalLM', 'MvpForCausalLM', 'OpenLlam

In [11]:
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0})

In [12]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [31]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [32]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
)

In [33]:
qa_chain("Generate 30 questions, chosen and rejected")

[INST] <>
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
<>


11.Retrieve
the
names
of
all
users
who
have
placed
orders
for
items
containing
'coffee'
in
the
contents:
12.List
all
orders
placed
by
users
who
have
'10'
in
their
contact
number:
13.Retrieve
the
names
of
users
who
have
placed
orders
for
items
with
a
price
higher
than
the
average
price
of
all
items:

3.Retrieve
the
names
of
all
users
who
have
not
placed
any
orders
yet:
4.Calculate
the
total
revenue
generated
from
all
orders
placed:
5.List
all
bills
along
with
the
corresponding
user's
name
and
address:

Question: Generate 30 questions, chosen and rejected
 

C:\Users\abhim\miniforge3\lib\site-packages\transformers\generation\configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\abhim\miniforge3\lib\site-packages\transformers\generation\configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


RuntimeError: Unexpected floating ScalarType in at::autocast::prioritize

## **RLHF**

In [34]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install peft
!pip install bitsandbytes
!pip install trl
!pip install tqdm

   ---------------------------------------- 0.0/101.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.6 MB 640.0 kB/s eta 0:02:39
   ---------------------------------------- 0.1/101.6 MB 744.7 kB/s eta 0:02:17
   ---------------------------------------- 0.1/101.6 MB 847.9 kB/s eta 0:02:00
   ---------------------------------------- 0.2/101.6 MB 1.2 MB/s eta 0:01:28
   ---------------------------------------- 0.3/101.6 MB 1.3 MB/s eta 0:01:20
   ---------------------------------------- 0.5/101.6 MB 1.5 MB/s eta 0:01:09
   ---------------------------------------- 0.6/101.6 MB 1.7 MB/s eta 0:01:01
   ---------------------------------------- 0.7/101.6 MB 1.9 MB/s eta 0:00:55
   ---------------------------------------- 0.9/101.6 MB 2.0 MB/s eta 0:00:51
   ---------------------------------------- 1.1/101.6 MB 2.2 MB/s eta 0:00:47
   ---------------------------------------- 1.1/101.6 MB 2.3 MB/s eta 0:

In [45]:
from datasets import load_dataset
from tqdm import tqdm

def prepare_sample_text(example):
    question = example.get('question', '')  # Get the value associated with the key 'question', or an empty string if the key is not found
    response_j = example.get('response_j', '')  # Get the value associated with the key 'response_j', or an empty string if the key is not found
    text = f"Question:{question}\n\nAnswer:{response_j}"
    return text


# def prepare_sample_text(example):
#     text=f"Question:{example['question']}\n\nAnswer:{example['response_j']}"
#     return text

def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens=0,0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text=prepare_sample_text(example)
        total_characters+=len(text)
        if tokenizer.is_fast:
            total_tokens+=len(tokenizer(text).tokens())
        else:
            total_tokens+=len(tokenizer.tokenize(text))
    return total_characters/total_tokens


# streaming=True

# dataset=load_dataset(
#     path="lvwerra/stack-exchange-paired",
#     data_dir="data/finetune", # the subset to use
#     split="train",
#     num_proc=4,
#     streaming=streaming,
# )

# dataset


from datasets import load_dataset

def load_custom_dataset(path, split="train", streaming=False):
    """
    Load a custom dataset either from a remote source or a local file.
    """
    if path.startswith("http://") or path.startswith("https://"):
        # Remote dataset
        return load_dataset(
            path=path,
            split=split,
            streaming=streaming
        )
    else:
        # Local file path
        import json
        data = []
        with open(path, "r", encoding="utf-8") as f:
            for line in f:
                data.append(json.loads(line))
        return data

streaming = True

path_to_dataset = "C:\\Users\\abhim\\Downloads\\preference_dataset.jsonl"

dataset = load_custom_dataset(
    path=path_to_dataset,
    split="train",
    streaming=streaming
)

dataset


[{'prompt': 'Please extract sender and receiver details from the document.',
  'chosen': 'Sender: John Mark, 23-B Main Street, Brooklyn, USA. Receiver: Steve Mark, 456 Elm Street, Oklahoma, USA.',
  'rejected': 'Unable to extract sender and receiver details due to document format.'},
 {'prompt': 'Capture flight details such as airline, flight number, departure, and arrival information.',
  'chosen': 'Airline: Delta Airlines. Flight number: DL123. Departure: New York JFK Airport. Arrival: London Heathrow Airport.',
  'rejected': 'Flight details missing from the document.'},
 {'prompt': 'Identify any special handling instructions mentioned in the document.',
  'chosen': 'Special handling instructions: Fragile - Handle with care.',
  'rejected': 'No special handling instructions provided.'},
 {'prompt': 'Extract description, quantity, and weight of goods being shipped.',
  'chosen': 'Goods description: Electronics. Quantity: 5 units. Weight: 20 kilograms.',
  'rejected': 'Description, qua

In [47]:
import random
if streaming:
    # Shuffle the dataset
    random.shuffle(dataset)
    
    # Number of prompts for validation set
    num_valid_prompts = min(400, len(dataset))  # Ensure not more than the total prompts
    
    # Number of prompts for training set
    num_train_prompts = len(dataset) - num_valid_prompts
    
    # Split the dataset into validation and training sets
    valid_data = dataset[:num_valid_prompts]
    train_data = dataset[num_valid_prompts:]


# if streaming:
#     valid_data=dataset.take(4000)
#     train_data=dataset.skip(4000)
#     train_data=train_data.shuffle(buffer_size=5000, seed=None)
else:
    dataset=dataset.train_test_split(test_size=0.005, seed=None)
    train_data=dataset["train"]
    valid_data=dataset["test"]

chars_per_token=chars_token_ratio(train_data, tokenizer)
chars_per_token

  3%|██▌                                                                           | 13/400 [00:00<00:00, 12881.16it/s]


2.5714285714285716

In [49]:
from trl.trainer import ConstantLengthDataset

train_dataset=ConstantLengthDataset(
    tokenizer,
    train_data,
    formatting_func=prepare_sample_text,
    infinite=True,
    seq_length=1024,
    chars_per_token=chars_per_token,
)

valid_dataset=ConstantLengthDataset(
    tokenizer,
    valid_data,
    formatting_func=prepare_sample_text,
    infinite=False,
    seq_length=1024,
    chars_per_token=chars_per_token,
)

In [61]:
import torch
from peft import AutoPeftModelForCausalLM
# from bitsandbytes import BitsAndBytesConfig

# bnb_config=BitsAndBytesConfig(
#     # It is used to enable 4-bit quantization by replacing the Linear layers with FP4/NF4 layers from bitsandbytes.
#     load_in_4bit=True,
#     # This sets the quantization data type in the bnb.nn.Linear4Bit layers.
#     bnb_4bit_quant_type="nf4",
#     # Support nested quantization
#     bnb_4bit_use_double_quant=True,
#     # This sets the computational type which might be different than the input time
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# base_model=AutoPeftModelForCausalLM.from_pretrained(
#     model_name,
#     # quantization_config=bnb_config,
#     device_map="auto",
#     torch_dtype=torch.bfloat16,
#     trust_remote_code=False,
# )

# base_model.config.use_cache=False
# print(base_model)

# Specify the name or path of the pre-trained model


base_model = AutoPeftModelForCausalLM.from_pretrained(
    model_name_or_path,  # Specify the model name or path
    # quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=False,
)

base_model.config.use_cache = False
print(base_model)






ValueError: Can't find 'adapter_config.json' at 'TheBloke/Llama-2-13B-chat-GPTQ'

In [ ]:
from peft LoraConfig, TaskType

peft_config=LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj","v_proj"],
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

peft_confg

In [ ]:
from trl import SFTTrainer


training_args=TrainingArguments(
    output_dir="./sft",
    max_steps=100,
    logging_steps=10,
    save_steps=10,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    group_by_length=False,
    learning_rate=1e-4,
    lr_scheduler_type="cosine",
    warmup_steps=50,
    weight_decay=0.05,
    optim="paged_adamw_32bit",
    fp16=True,
    remove_unused_columns=False,
)

sft_trainer=SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    packing=True,
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_args,
)

sft_trainer.train()

In [ ]:
import gc

del sft_trainer, base_model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
model=AutoPeftModelForcausalLM.from_pretrained("/kaggle/working/sft/final_checkpoint", device_map="auto", torch_dtype=torch.bfloat16)
model=model.merge_and_unload()

model.save_pretrained("/kaggle/working/sft/final_merged_checkpoint", safe_serialization=True)

In [ ]:
def return_prompt_and_responses(samples)-> Dict[str,str]:
    return {
        "prompt":[
            "Question:"+question+"\n\nAnswer:" for question in samples["question"]
        ],
        "chosen": samples["response_j"],
        "rejected": samples["response_k"],
    }


def get_stack_exchange_paired(data_dir="data/rl",sanity_check=False,cache_dir=None,num_proc=24):
    dataset=load_dataset(
        "lvwerra/stack-exchange-paired",
        split="train",
        data_dir="data/rl",
        cache_dir=cache_dir,
    )
    original_columns=dataset.column_names
    
    if sanity_check:
        dataset=dataset.select(range(min(len(dataset), 1000)))
    
    return dataset.map(
        return_prompt_and_responses,
        batched=True,
        num_proc=num_proc,
        remove_columns=original_columns,
    )

In [ ]:
model=AutoModelForCausalLM.from_pretrained(
    "/kaggle/working/sft/final_merged_checkpoint",
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

model.config.use_cache=False

model_ref=AutoModelForCausalLM.from_pretrained(
    "/kaggle/working/sft/final_merged_checkpoint",
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)

In [ ]:
tokenizer.pad_token=tokenizer.eos_token

In [ ]:
train_dataset=get_stack_exchange_paired(
    data_dir="data/rl",
    sanity_check=False
)

train_dataset=train_dataset.filter(
    lambda x: len(x["prompt"])+len(x["chosen"])<=1024 and len(x["prompt"])+len(x["rejected"])<=1024
)

eval_dataset=get_stack_exchange_paired(data_dir="data/evaluation", sanity_check=True)
eval_dataset=eval_dataset.filter(
    lambda x: len(x["prompt"])+len(x["chosen"])<=1024 and len(x["prompt"])+len(x["rejected"])<=1024
)

In [ ]:
training_args=TrainingArguments(
    per_device_train_batch_size=4,
    per_device_eval_batch_size=1,
    max_steps=1000,
    logging_steps=10,
    save_steps=100,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-4,
    evaluation_strategy="steps",
    eval_steps=100,
    report_to="wandb",
    lr_scheduler_type="cosine",
    warmup_steps=100,
    optim="paged_adamw_32bit",
    bf16=True,
    remove_unused_columns=False,
)

peft_config=LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=['q_proj','v_proj','k_proj','out_proj','fc_in','fc_out','wte',],
    bias="none",
    task_type="CAUSAL_LM",
)

dpo_trainer=DPOTrainer(
    model,
    model_ref,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    max_prompt_length=512,
    max_length=1024,
)

dpo_trainer.train()